<a href="https://colab.research.google.com/github/pasumarthi/EIP4/blob/master/Session5/PersonAttrubutes_ipynb_v7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
# mount gdrive and unzip data
from google.colab import drive
drive.mount('/content/gdrive')
!unzip -q "/content/gdrive/My Drive/EIP3.0/hvc_data.zip"
# look for `hvc_annotations.csv` file and `resized` dir
%ls 

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
replace resized/9733.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: N
gdrive/  hvc_annotations.csv  resized/  sample_data/  saved_models/


In [0]:
%tensorflow_version 1.x

import cv2
import json

import numpy as np
import pandas as pd

from functools import partial
from pathlib import Path 
from tqdm import tqdm

from google.colab.patches import cv2_imshow

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder


from keras.applications import VGG16
from keras.layers.core import Dropout
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.layers import Input
from keras.models import Model
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator


In [28]:
# load annotations
df = pd.read_csv("hvc_annotations.csv")
del df["filename"] # remove unwanted column
df.head()

,gender,imagequality,age,weight,carryingbag,footwear,emotion,bodypose,image_path
0,male,Average,35-45,normal-healthy,Grocery/Home/Plastic Bag,Normal,Neutral,Front-Frontish,resized/1.jpg
1,female,Average,35-45,over-weight,None,Normal,Angry/Serious,Front-Frontish,resized/2.jpg
2,male,Good,45-55,normal-healthy,Grocery/Home/Plastic Bag,CantSee,Neutral,Front-Frontish,resized/3.jpg
3,male,Good,45-55,normal-healthy,Daily/Office/Work Bag,Normal,Neutral,Front-Frontish,resized/4.jpg
4,female,Good,35-45,slightly-overweight,None,CantSee,Neutral,Front-Frontish,resized/5.jpg


In [29]:
# one hot encoding of labels

one_hot_df = pd.concat([
    df[["image_path"]],
    pd.get_dummies(df.gender, prefix="gender"),
    pd.get_dummies(df.imagequality, prefix="imagequality"),
    pd.get_dummies(df.age, prefix="age"),
    pd.get_dummies(df.weight, prefix="weight"),
    pd.get_dummies(df.carryingbag, prefix="carryingbag"),
    pd.get_dummies(df.footwear, prefix="footwear"),
    pd.get_dummies(df.emotion, prefix="emotion"),
    pd.get_dummies(df.bodypose, prefix="bodypose"),
], axis = 1)

one_hot_df.head().T

,0,1,2,3,4
image_path,resized/1.jpg,resized/2.jpg,resized/3.jpg,resized/4.jpg,resized/5.jpg
gender_female,0,1,0,0,1
gender_male,1,0,1,1,0
imagequality_Average,1,1,0,0,0
imagequality_Bad,0,0,0,0,0
imagequality_Good,0,0,1,1,1
age_15-25,0,0,0,0,0
age_25-35,0,0,0,0,0
age_35-45,1,1,0,0,1
age_45-55,0,0,1,1,0


In [0]:
import keras
import numpy as np
from keras.preprocessing.image import img_to_array

# Label columns per attribute
_gender_cols_ = [col for col in one_hot_df.columns if col.startswith("gender")]
_imagequality_cols_ = [col for col in one_hot_df.columns if col.startswith("imagequality")]
_age_cols_ = [col for col in one_hot_df.columns if col.startswith("age")]
_weight_cols_ = [col for col in one_hot_df.columns if col.startswith("weight")]
_carryingbag_cols_ = [col for col in one_hot_df.columns if col.startswith("carryingbag")]
_footwear_cols_ = [col for col in one_hot_df.columns if col.startswith("footwear")]
_emotion_cols_ = [col for col in one_hot_df.columns if col.startswith("emotion")]
_bodypose_cols_ = [col for col in one_hot_df.columns if col.startswith("bodypose")]

class PersonDataGenerator(keras.utils.Sequence):
    def __init__(self, df, batch_size=32, shuffle=True, aug_list=[], incl_orig=True):
        self.df = df
        self.batch_size=batch_size
        self.shuffle = shuffle
        self.on_epoch_end()
        
        self.aug_list = aug_list
        self.incl_orig = incl_orig
        self.orig_len = int(np.floor(self.df.shape[0] / self.batch_size))

    def __len__(self):
        if self.incl_orig:
            delta = 1
        else:
            delta = 0
        return self.orig_len * (len(self.aug_list) + delta)

    def __getitem__(self, index):
        if not self.incl_orig :
            index += self.orig_len - 1

        if index > self.orig_len - 1:
            aug = self.aug_list[index // self.orig_len - 1]
            index %= self.orig_len
        else:
            aug = None

        batch_slice = slice(index * self.batch_size, (index + 1) * self.batch_size)
        items = self.df.iloc[batch_slice]
        
        images = np.stack([cv2.imread(item["image_path"]) for _, item in items.iterrows()])
        images = images / 255.0
#        print(images)      
        
        if aug is not None:
            images = aug.flow(images, shuffle=False).next()
        
        target = {
            "gender_output": items[_gender_cols_].values,
            "image_quality_output": items[_imagequality_cols_].values,
            "age_output": items[_age_cols_].values,
            "weight_output": items[_weight_cols_].values,
            "bag_output": items[_carryingbag_cols_].values,
            "pose_output": items[_bodypose_cols_].values,
            "footwear_output": items[_footwear_cols_].values,
            "emotion_output": items[_emotion_cols_].values,
        }
        
        return images, target

    def on_epoch_end(self):
        """Updates indexes after each epoch"""
        if self.shuffle == True:
            self.df = self.df.sample(frac=1).reset_index(drop=True)

In [32]:
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(one_hot_df, test_size=0.15)
train_df.shape, val_df.shape

((11537, 28), (2036, 28))

In [30]:
train_df.head()

,image_path,gender_female,gender_male,imagequality_Average,imagequality_Bad,imagequality_Good,age_15-25,age_25-35,age_35-45,age_45-55,age_55+,weight_normal-healthy,weight_over-weight,weight_slightly-overweight,weight_underweight,carryingbag_Daily/Office/Work Bag,carryingbag_Grocery/Home/Plastic Bag,carryingbag_None,footwear_CantSee,footwear_Fancy,footwear_Normal,emotion_Angry/Serious,emotion_Happy,emotion_Neutral,emotion_Sad,bodypose_Back,bodypose_Front-Frontish,bodypose_Side
721,resized/722.jpg,0,1,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0
6195,resized/6196.jpg,0,1,0,1,0,0,0,1,0,0,1,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,1
10986,resized/10988.jpg,0,1,0,1,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0
4178,resized/4179.jpg,0,1,0,0,1,0,1,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,1,0,0,1,0
4223,resized/4224.jpg,0,1,1,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,1,0,0,1,0,0,1,0


In [0]:
def get_random_eraser(p=0.5, s_l=0.02, s_h=0.4, r_1=0.3, r_2=1/0.3, v_l=0, v_h=255, pixel_level=False):
    def eraser(input_img):
        img_h, img_w, img_c = input_img.shape
        p_1 = np.random.rand()

        if p_1 > p:
            return input_img

        while True:
            s = np.random.uniform(s_l, s_h) * img_h * img_w
            r = np.random.uniform(r_1, r_2)
            w = int(np.sqrt(s / r))
            h = int(np.sqrt(s * r))
            left = np.random.randint(0, img_w)
            top = np.random.randint(0, img_h)

            if left + w <= img_w and top + h <= img_h:
                break

        if pixel_level:
            c = np.random.uniform(v_l, v_h, (h, w, img_c))
        else:
            c = np.random.uniform(v_l, v_h)

        input_img[top:top + h, left:left + w, :] = c

        return input_img

    return eraser

In [0]:
from keras.callbacks import LearningRateScheduler
def scheduler(epoch, lr):
   return round(0.003 * 1/(1 + 0.319 * epoch), 10)
   
lr_scheduler = LearningRateScheduler(scheduler)  


In [0]:
import os
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from keras.models import Model
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'PersonAttribute%s_model.{epoch:03d}.h5'
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath = os.path.join(save_dir, model_name)

# Prepare callbacks for model saving and for learning rate adjustment.
checkpoint = ModelCheckpoint(filepath=filepath,
                             monitor='val_loss',
                             verbose=1,
                             mode='min',
                             save_best_only=True)

lr_scheduler = LearningRateScheduler(scheduler)

lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                                cooldown=0,
                                patience=5,
                                min_lr=0.5e-6)

#callbacks = [checkpoint, lr_reducer, lr_scheduler]
callbacks = [checkpoint, lr_scheduler]

In [0]:
train_gen = PersonDataGenerator(
    train_df, 
    batch_size=32, 
    aug_list=[
        ImageDataGenerator(horizontal_flip=True,vertical_flip=True,shear_range=0.2,
        zoom_range=0.2,preprocessing_function=get_random_eraser(v_l=0, v_h=1, pixel_level=True))
          ],
    incl_orig=False,
)
valid_gen = PersonDataGenerator(val_df, batch_size=64, shuffle=False)

In [39]:
# get number of output units from data
images, targets = next(iter(train_gen))
num_units = { k.split("_output")[0]:v.shape[1] for k, v in targets.items()}
num_units

{'age': 5,
 'bag': 3,
 'emotion': 4,
 'footwear': 3,
 'gender': 2,
 'image_quality': 3,
 'pose': 3,
 'weight': 4}

In [0]:
backbone = keras.applications.resnet_v2.ResNet50V2(
    weights=None, 
    include_top=False, 
    pooling='avg',
    input_tensor=Input(shape=(224, 224, 3))
)

neck = backbone.output
#neck = Flatten(name="flatten")(neck)
neck = Dense(512, activation="relu")(neck)


def build_tower(in_layer):
    #neck = Dropout(0.2)(in_layer)
    neck = Dense(128, activation="relu")(in_layer)
    #neck = Dropout(0.3)(neck)
    #neck = Dense(128, activation="relu")(neck)
    return neck


def build_head(name, in_layer):
    return Dense(
        num_units[name], activation="sigmoid", name=f"{name}_output"
    )(in_layer)

# heads
gender = build_head("gender", build_tower(neck))
image_quality = build_head("image_quality", build_tower(neck))
age = build_head("age", build_tower(neck))
weight = build_head("weight", build_tower(neck))
bag = build_head("bag", build_tower(neck))
footwear = build_head("footwear", build_tower(neck))
emotion = build_head("emotion", build_tower(neck))
pose = build_head("pose", build_tower(neck))


model = Model(
    inputs=backbone.input, 
    outputs=[gender, image_quality, age, weight, bag, footwear, pose, emotion]
)

In [45]:
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_5[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
pool1_pad (ZeroPadding2D)       (None, 114, 114, 64) 0           conv1_conv[0][0]                 
____________________________________________________________________________________________

In [46]:
from keras.optimizers import Adam 
#losses = {
# 	"gender_output": "binary_crossentropy",
# 	"image_quality_output": "categorical_crossentropy",
# 	"age_output": "categorical_crossentropy",
# 	"weight_output": "categorical_crossentropy",

# }
loss_weights = {"gender_output": 1.0, "image_quality_output": 2.0, "age_output": 2.0,"emotion_output":1.5,"footwear_output":1,"weight_output":1.5,"pose_output":2,"bag_output":1}
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

model.compile(
    optimizer=sgd,
    loss="binary_crossentropy", 
    loss_weights=loss_weights, 
    metrics=["accuracy"]
)
'''model.compile(loss='categorical_crossentropy',
              optimizer=Adam(0.01),
              metrics=['accuracy'])'''

"model.compile(loss='categorical_crossentropy',\n              optimizer=Adam(0.01),\n              metrics=['accuracy'])"

In [47]:
model.fit_generator(
    generator=train_gen,
    validation_data=valid_gen,
    use_multiprocessing=True,
    workers=5, 
    epochs=50,
    verbose=1,callbacks=callbacks
)

Epoch 1/50
359/360 [============================>.] - ETA: 0s - loss: 6.2336 - gender_output_loss: 0.6813 - image_quality_output_loss: 0.5792 - age_output_loss: 0.4599 - weight_output_loss: 0.4275 - bag_output_loss: 0.5514 - footwear_output_loss: 0.5941 - pose_output_loss: 0.5496 - emotion_output_loss: 0.3920 - gender_output_acc: 0.5634 - image_quality_output_acc: 0.6973 - age_output_acc: 0.7989 - weight_output_acc: 0.8127 - bag_output_acc: 0.7064 - footwear_output_acc: 0.6756 - pose_output_acc: 0.7443 - emotion_output_acc: 0.8531

360/360 [==============================] - 213s 591ms/step - loss: 6.2328 - gender_output_loss: 0.6814 - image_quality_output_loss: 0.5793 - age_output_loss: 0.4598 - weight_output_loss: 0.4271 - bag_output_loss: 0.5513 - footwear_output_loss: 0.5943 - pose_output_loss: 0.5497 - emotion_output_loss: 0.3918 - gender_output_acc: 0.5632 - image_quality_output_acc: 0.6973 - age_output_acc: 0.7989 - weight_output_acc: 0.8130 - bag_output_acc: 0.7065 - footwear_ou

/usr/local/lib/python3.6/dist-packages/keras/utils/data_utils.py:610: UserWarning: The input 49 could not be retrieved. It could be because a worker has died.
  UserWarning)


360/360 [==============================] - 221s 613ms/step - loss: 5.4556 - gender_output_loss: 0.4885 - image_quality_output_loss: 0.5369 - age_output_loss: 0.4439 - weight_output_loss: 0.4109 - bag_output_loss: 0.5114 - footwear_output_loss: 0.5000 - pose_output_loss: 0.4089 - emotion_output_loss: 0.3732 - gender_output_acc: 0.7557 - image_quality_output_acc: 0.7180 - age_output_acc: 0.8006 - weight_output_acc: 0.8199 - bag_output_acc: 0.7429 - footwear_output_acc: 0.7538 - pose_output_acc: 0.8142 - emotion_output_acc: 0.8546 - val_loss: 5.5691 - val_gender_output_loss: 0.5085 - val_image_quality_output_loss: 0.5601 - val_age_output_loss: 0.4407 - val_weight_output_loss: 0.4098 - val_bag_output_loss: 0.5237 - val_footwear_output_loss: 0.5144 - val_pose_output_loss: 0.4304 - val_emotion_output_loss: 0.3636 - val_gender_output_acc: 0.7412 - val_image_quality_output_acc: 0.7011 - val_age_output_acc: 0.8017 - val_weight_output_acc: 0.8206 - val_bag_output_acc: 0.7270 - val_footwear_outpu

Process ForkPoolWorker-746:
Process ForkPoolWorker-755:
Process ForkPoolWorker-748:
Process ForkPoolWorker-754:
Process ForkPoolWorker-745:
KeyboardInterrupt
Process ForkPoolWorker-747:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  Fi

KeyboardInterrupt: ignored